In [1]:
root = r'C:\Users\akila\Downloads\fine-tune-MaskRcnn-master\beagle'

In [2]:
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image
import json

In [3]:
class BeagleDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir , transforms = None):
        self.data_dir = data_dir
        self.transforms = transforms

        #load the annotations file, it also contain information of image names
        # load the annotations
        annotations1 = json.load(open(os.path.join(data_dir, 'via_region_data.json')))
        self.annotations = list(annotations1.values())

    def __getitem__(self,idx):
        #get the image path from the annotations data
        img_name = self.annotations[idx]['filename']
        img_path = os.path.join(self.data_dir,img_name)
        img = Image.open(img_path).convert("RGB")

        #first id is the background , objects count from 1
        obj_ids = np.array(range(len(self.annotations[idx]['regions']))) + 1

        #get bounding box coordinates for each objects
        num_objs = len(obj_ids)
        boxes = []

        for i in range(num_objs):
            xmin = np.min(self.annotations[idx]["regions"][i]["shape_attributes"]["all_points_x"])
            xmax = np.max(self.annotations[idx]["regions"][i]["shape_attributes"]["all_points_x"])
            ymin = np.min(self.annotations[idx]["regions"][i]["shape_attributes"]["all_points_y"])
            ymax = np.max(self.annotations[idx]["regions"][i]["shape_attributes"]["all_points_y"])
            boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.as_tensor(boxes, dtype = torch.float32)

        labels = torch.ones((num_objs,),dtype = torch.int64)

        image_id = torch.tensor([idx])
        area = (boxes[:,3] - boxes[:,1]) * (boxes[:,2] - boxes[:,0])

        iscrowd = torch.zeros((num_objs, ), dtype = torch.int64)

        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = image_id
        target['area'] = area
        target['iscrowd'] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.annotations)

In [4]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

def build_model(num_classses):
    #load the instance segmentation

    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)

    #get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model
    

In [5]:
# dataset = BeagleDataset(r'C:\Users\akilarasan.p\Downloads\fine-tune-MaskRcnn-master\beagle\train', get_transform(train= True))
# dataset_test = BeagleDataset(r'C:\Users\akilarasan.p\Downloads\fine-tune-MaskRcnn-master\beagle\val', get_transform(train= False))

In [6]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T

def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

# use our dataset and defined transformations
dataset = BeagleDataset(r'C:\Users\akila\Downloads\fine-tune-MaskRcnn-master\beagle\train', get_transform(train=True))
dataset_test = BeagleDataset(r'C:\Users\akila\Downloads\fine-tune-MaskRcnn-master\beagle\val', get_transform(train=False))

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=0,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=0,
    collate_fn=utils.collate_fn)

In [7]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [8]:
# our dataset has two classes only - background and beagle
num_classes = 2

# get the model using our helper function
model = build_model(num_classes)
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

C:\Users\akila\anaconda3\envs\gpu\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\akila\anaconda3\envs\gpu\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
pip install six

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [10]:
# number of epochs
num_epochs = 10
 
for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

Epoch: [0]  [ 0/38]  eta: 0:01:32  lr: 0.000140  loss: 0.8723 (0.8723)  loss_classifier: 0.7261 (0.7261)  loss_box_reg: 0.1294 (0.1294)  loss_objectness: 0.0001 (0.0001)  loss_rpn_box_reg: 0.0166 (0.0166)  time: 2.4258  data: 0.0550  max mem: 2248
Epoch: [0]  [10/38]  eta: 0:00:28  lr: 0.001490  loss: 0.3675 (0.5223)  loss_classifier: 0.2183 (0.3624)  loss_box_reg: 0.1452 (0.1458)  loss_objectness: 0.0013 (0.0034)  loss_rpn_box_reg: 0.0070 (0.0108)  time: 1.0148  data: 0.1369  max mem: 3166
Epoch: [0]  [20/38]  eta: 0:00:21  lr: 0.002840  loss: 0.2942 (0.3900)  loss_classifier: 0.1355 (0.2453)  loss_box_reg: 0.1322 (0.1288)  loss_objectness: 0.0016 (0.0032)  loss_rpn_box_reg: 0.0061 (0.0127)  time: 1.1440  data: 0.1515  max mem: 3436
Epoch: [0]  [30/38]  eta: 0:00:11  lr: 0.004190  loss: 0.1946 (0.3245)  loss_classifier: 0.0755 (0.1836)  loss_box_reg: 0.1084 (0.1266)  loss_objectness: 0.0016 (0.0029)  loss_rpn_box_reg: 0.0061 (0.0114)  time: 1.7109  data: 0.1623  max mem: 3436
Epoch: [

In [13]:
# set to evaluation mode
torch.save(model, 'faster-rcnn-beagle22.pt')
model.eval()
CLASS_NAMES = ['__background__', 'beagle']
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [12]:
from six import string_classes


ImportError: cannot import name 'string_classes' from 'six' (C:\Users\akila\anaconda3\envs\gpu\Lib\site-packages\six.py)